# Merge the stuff

In [1]:
import pandas as pd
from os import *
from pathlib import *
from matplotlib import *
import numpy as np

In [2]:
def clean_csv(the_file,reading_type):
    frame = pd.read_csv(the_file,header=None)
    multi_indexing = frame.iloc[[0, 1], 1:].apply(lambda x: (f'{int(x.iloc[0]):06}', f'{int(x.iloc[1]):06}'))
    multi_indexing = pd.MultiIndex.from_tuples(multi_indexing, names=['easting', 'northing'])
    frame = frame.drop([0,1],axis=0)

    date_index = frame.iloc[0:,0].apply(lambda x: pd.to_datetime(x))
    #date_index = pd.MultiIndex.from_tuples(date_index, names = ['date','reading'])
    frame = frame.drop(0,axis=1)
    frame = frame.set_index(date_index)
    frame.index.name = 'date'
    frame.columns = multi_indexing
    frame = frame.loc['2000-01-01':]
    return frame

## Rainfall

In [3]:
directory = [k for k in os.listdir('./rainfall') if k.endswith('csv')]
to_append = []
for file in directory[0:]:
    to_append.append(clean_csv('./rainfall/' + file,'rainfall'))
rainfall = pd.concat(to_append,axis=1,verify_integrity=True)

## Mean Temp

In [4]:
directory = [k for k in os.listdir('./mean_temp') if k.endswith('csv')]
to_append = []
for file in directory[0:]:
    to_append.append(clean_csv('./mean_temp/' + file,'mean_temp'))
mean_temp = pd.concat(to_append,axis=1,verify_integrity=True)

## Max Temp

In [5]:
directory = [k for k in os.listdir('./max_temp') if k.endswith('csv')]
to_append = []
for file in directory[0:]:
    to_append.append(clean_csv('./max_temp/' + file,'max_temp'))
max_temp = pd.concat(to_append,axis=1,verify_integrity=True)

## Min Temp

In [6]:
directory = [k for k in os.listdir('./min_temp') if k.endswith('csv')]
to_append = []
for file in directory[0:]:
    to_append.append(clean_csv('./min_temp/' + file,'min_temp'))
min_temp = pd.concat(to_append,axis=1,verify_integrity=True)

In [38]:
rainfall = rainfall.loc[:,rainfall.columns & min_temp.columns]
rainfall = rainfall.loc[:,np.intersect1d(rainfall.columns, min_temp.columns)]
rainfall = rainfall.loc[:,rainfall.columns.intersection(min_temp.columns)]

## Merge the stuff

In [7]:
#THIS ONE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
weather = pd.concat([min_temp,mean_temp,max_temp,rainfall],keys=['min_temp','mean_temp','max_temp','rainfall'],axis=0,join='outer')

In [8]:
# Set about fixing the indexes
weather.index.rename('reading',level=0,inplace=True)

In [9]:
weather.index = weather.index.swaplevel(0)

In [10]:
weather.sort_index(axis=0,level=0,inplace=True)

In [11]:
weather

easting              002500        007500               012500                \
northing             522500 527500 517500 522500 527500 512500 517500 522500   
date       reading                                                             
2000-01-01 max_temp    9.31   9.17   8.87   8.31   8.92   8.45   8.24   7.57   
           mean_temp   6.22   6.08   5.86   5.83   5.88   4.92   4.79   4.46   
           min_temp    3.13   2.99   2.85   3.36   2.84   1.39   1.33   1.34   
           rainfall    0.33   0.21   0.38   0.00   0.18   0.12   0.41   0.42   
2000-01-02 max_temp    9.27   9.06   8.85   8.02   8.86   8.68   8.40   7.53   
           mean_temp   6.48   6.34   6.14   5.94   6.19   5.54   5.31   4.77   
           min_temp    3.70   3.61   3.43   3.85   3.51   2.39   2.23   2.01   
           rainfall    2.58   2.24   3.04   2.38   2.43   1.71   2.49   2.88   
2000-01-03 max_temp    7.38   7.40   6.80   6.67   7.22   6.69   6.47   5.60   
           mean_temp   6.31   6.18   5.80   5.58   5.96   5.23   4.96   4.21   
           min_temp    5.24   4.95   4.79   4.50   4.70   3.77   3.45   2.81   
           rainfall    2.26   2.69   2.77   3.75   3.30   3.22   3.50   4.09   
2000-01-04 max_temp    8.68   8.22   8.19   5.92   7.78   8.08   7.71   6.59   
           mean_temp   5.27   5.10   4.87   4.31   4.88   4.47   4.24   3.60   
           min_temp    1.86   1.97   1.55   2.71   1.97   0.86   0.77   0.61   
           rainfall    1.51   1.72   1.84   2.56   2.15   1.96   2.22   2.66   
2000-01-05 max_temp   10.66  10.67  10.22  10.36  10.57  10.02   9.82   9.11   
           mean_temp   5.76   5.76   5.35   5.37   5.62   5.05   4.83   4.22   
           min_temp    0.86   0.85   0.47   0.39   0.66   0.09  -0.16  -0.67   
           rainfall    7.83   8.72   9.50  12.24  10.65   8.46  11.07  13.30   
2000-01-06 max_temp    9.63   9.25   9.05   8.59   8.91   7.79   7.23   6.36   
           mean_temp   6.50   6.10   6.02   5.64   5.82   4.72   4.29   3.58   
           min_temp    3.37   2.94   2.98   2.69   2.73   1.65   1.36   0.80   
           rainfall    1.52   2.01   1.93   2.86   2.51   4.41   2.96   3.14   
2000-01-07 max_temp   10.74  10.78  10.23  10.45  10.67  10.02   9.85   9.10   
           mean_temp   6.80   6.86   6.32   6.80   6.73   5.84   5.62   5.07   
           min_temp    2.86   2.94   2.42   3.14   2.79   1.66   1.39   1.03   
           rainfall    6.67   6.29   7.98   6.54   6.90   7.64   8.02   8.68   
2000-01-08 max_temp    6.15   5.94   5.59   5.46   5.77   4.88   4.63   3.77   
           mean_temp   4.68   4.54   4.14   4.12   4.35   3.47   3.22   2.47   
...                     ...    ...    ...    ...    ...    ...    ...    ...   
2016-12-24 min_temp    3.19   3.19   2.79   2.85   3.11   2.50   2.32   1.65   
           rainfall    5.37   5.13   6.33   5.72   5.67   6.89   6.40   6.98   
2016-12-25 max_temp   13.12  13.10  12.57  12.40  12.91  12.26  12.03  11.20   
           mean_temp   9.15   9.13   8.71   8.66   9.03   8.44   8.24   7.51   
           min_temp    5.18   5.16   4.86   4.92   5.15   4.63   4.45   3.83   
           rainfall    5.21   5.38   5.66   5.24   6.04   8.76   7.07   8.23   
2016-12-26 max_temp    6.64   6.66   6.10   5.96   6.52   5.80   5.61   4.80   
           mean_temp   4.73   4.75   4.22   4.19   4.61   3.89   3.70   2.97   
           min_temp    2.83   2.84   2.34   2.42   2.71   1.97   1.79   1.13   
           rainfall    0.15   0.26   0.02   0.05   0.31   1.16   0.44   0.58   
2016-12-27 max_temp    7.02   6.99   6.48   6.45   6.75   6.05   6.00   5.55   
           mean_temp   4.54   4.62   4.09   4.16   4.39   3.69   3.64   3.25   
           min_temp    2.07   2.25   1.69   1.88   2.04   1.33   1.29   0.95   
           rainfall    0.02   0.02   0.00   0.00   0.02   0.34   0.05   0.02   
2016-12-28 max_temp    6.71   6.66   6.42   6.34   6.67   6.27   6.14   5.60   
           mean_temp   3.41   3.52   3.24   3.37   3.52   3.07   3.07   2.84   


In [12]:
weather.to_csv('met_data_complete.csv')